In [2]:
from sage.groups.matrix_gps.heisenberg import HeisenbergGroup
from sage.algebras.lie_algebras.heisenberg import HeisenbergAlgebra

In [10]:
def lie_algebra_to_cdga(L):
    """
    Construct a differential graded commutative algebra from the 
    underlying dual vector space to ``L``by calculating the differential 
    as the adjoint of the Lie bracket.
    
    INPUT:

    - ``L`` -- a Lie algebra
    
    EXAMPLES::
        sage: L.<X,Y,Z,W> = LieAlgebra(QQ, {('X','Y'): {'Z': 1}}, nilpotent=True)
        sage: A = lie_algebra_to_cdga(L); A
        Commutative Differential Graded Algebra with generators ('X', 'Y', 'Z', 'W') 
        in degrees (1, 1, 1, 1) over Rational Field with differential:
           X --> 0
           Y --> 0
           Z --> X*Y
           W --> 0
    
    ALGORITHM:
    Create the graded commutative algebra 'A' with the same variable names 
    as ``L``, and all generators are of degree 1. For each pair of generators 
    of ``L``, record their bracket as a coefficient vector. For each generator 
    of `A`, construct the differential as a linear combination of wedge products 
    by taking inner product with each coefficient vectors.

    """
    gens_to_vectors= {}
    brackets_to_vectors = {}
    cdga_generators = {}
    A = GradedCommutativeAlgebra(QQ, names=L.basis())
    A.inject_variables(verbose=False)
    for index, gen in enumerate(L.basis()):
        gens_to_vectors[A.gen(index)]=gen.to_vector()
    for i in range(len(L.gens())):
        for j in range(i+1, len(L.gens())):
            brackets_to_vectors[(A.gen(i),A.gen(j))]=L.gen(i).bracket(L.gen(j)).to_vector()
    for k,v in gens_to_vectors.items():
        combination = 0 
        for key, value in brackets_to_vectors.items():
            coef = v.dot_product(value)
            if  coef != 0:
                wedge = 1
                for bracket in key:
                    wedge = wedge * bracket
                wedge = coef * wedge
                combination = combination + wedge
        cdga_generators[k]=combination
    return A.cdg_algebra(cdga_generators)

In [11]:
def find_formality(B): 
    """
    Compute the highest degree `D` for which the map from ``B`` to the exterior
    algebra on H^1(B) is surjective on cohomology. Return `D`-2, which agrees
    with the degree of formality when ``B`` is the dual of 2-step nilpotent
    Lie algebra.
    
    INPUT:

    - ``B`` -- a commutative differential graded algebra

    EXAMPLES::
        sage: A.<e1, e2, e3, e4, e5> = GradedCommutativeAlgebra(QQ)
        sage: B = A.cdg_algebra({e5 : e1*e2 + e3*e4})
        sage: find_formality(B)
        1
    
    ALGORITHM:
    Reduce a basis of the `n'-th cohomology modulo all the degree `n`
    products of the lower degree cohomologies. If such basis is non-empty,
    then `D`=`n` is found.
    
    NOTE: This uses a modified version of cohomology_generators()
    in 'commutative_dga.py'.
    
    """
    max_degree = floor(len(B.cohomology(1).gens()) / 2) + 2
    smaller_degree = {i: [g.representative() for g in B.cohomology(i).basis().keys()] for i in range(1, max_degree)}
    for degree in range(1, max_degree):
        already_generated = set()
        for i in range(1, degree):
            for a in smaller_degree[i]:
                for b in smaller_degree[degree - i]:
                    already_generated.add(a * b)
        CR = B.cohomology_raw(degree)
        V = CR.V()
        S = CR.submodule([CR(V(g.basis_coefficients(total=True))) for g in already_generated if not g.is_zero()])
        Q = CR.quotient(S)
        if degree > 1 and Q.basis():
            return degree - 2